# Demonstration of the Earthquake Catalogue Tools

The earthquake catalogue toolkit <b>OQ-EQCAT</b> contains tools for data mining earthquake databases for the ultimate aim of creating a magnitude homogeneous earthquake catalogue.

In this tutorial we will do the following:

1. Build an earthquake catalogue database using data from the International Seismological Centre (www.isc.ac.uk)

2. Apply various selection criteria

3. Explore different catalogue/agency regressions

4. Select several agencies and apply a homogenisation

# Building the database

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import eqcat.parsers.isf_catalogue_reader as icr
import eqcat.catalogue_query_tools as cqt

We are interested only in the events that the reporting networks believe to be natural tectonic events.

Using the descriptions reported in the contributing agencies we remove events with descriptions containing specific keywords indicating anthropogenic origin.

In [ ]:
raw_file = "input_data/catalogues/catalogue_input_raw.txt"
rejection_keywords = ["mining", "geothermal", "explosion", "quarry", "reservoir", "rockburst"]
reader = icr.ISFReader(raw_file, rejection_keywords=rejection_keywords)
catalogue = reader.read_file("FRC", "ISC (France)")

We keep those events identified as anthropogenic in a separate database stored in the reader class.

Which events have been rejected and why.

In [ ]:
for event in reader.rejected_catalogue.events:
    print event.id, event.description, event.comment

Now we can build the database and store it to file - we only need to do this once!

In [ ]:
# Build HDF5 database
CAT_DB_FILE = "output_data/catalogue_db.hdf5"
_ = catalogue.build_dataframe(CAT_DB_FILE)
print "Comment this section out after first use!"

From now on we can simply load in the data from the database

In [ ]:
CAT_DB_FILE = "output_data/catalogue_db.hdf5"
db1 = cqt.CatalogueDB(CAT_DB_FILE)

#### Origin Table

In [ ]:
db1.origins

#### Magnitude Table

In [ ]:
db1.magnitudes

## Visualising the Catalogue

View the whole catalogue ...

In [ ]:
# Set up the configuration of the limits
map_config = {"llon": -10.0, "ulon": 15., "llat": 35.0, "ulat": 55.0, "parallel": 5.0, "meridian": 5.0,
              "resolution": "l"}
cqt.plot_catalogue_map(map_config, db1)

Choose a particular agency within the catalogue ... then show it

In [ ]:
selector = cqt.CatalogueSelector(db1)
madrid_catalogue = selector.limit_to_agency("MDD")
cqt.plot_catalogue_map(map_config, madrid_catalogue)

Choose a particular sub-region of the catalogue ... then show it

In [ ]:
selector = cqt.CatalogueSelector(db1)
central_france = selector.select_within_bounding_box([0., 45., 5., 50.])
cqt.plot_catalogue_map(map_config, central_france)

## Exploring the catalogue

We could begin to execute queries on the catalogue based on what we are hoping should be there.

But to help us understand which queries might be more likely to return significant results it can be useful to know how many events are reported by particular agencies and in which magnitude scales.

There is a tool for this:

In [ ]:
agency_magnitude_stats = cqt.get_agency_magtype_statistics(db1)

Some notes:

* `LDG` - Laboratoire de Détection et de Géophysique/CEA

* `STR` - is Institut de Physique du Globe (France)

* `CSEM` - Euro-Mediterranean Seismological Centre (EMSC-CSEM)

* `MDD` - Instituto Geográfico Nacional (Madrid, Spain)

* `ROM` - INGV, Italy

* `ZUR` - Swiss Seismological Service (SED, ETH Zurich)

* `GEN` - Dipartimento per lo Studio del Territorio e delle sue Risorse (RSNI)

* `BGR` - BRGM, Orleans France


### Executing Queries

To find all of the events with magnitudes reported by specific pairs of agencies we can simply execute the function below.

In this example we ask to find all of the events with a reported local magnitude $M_L$ from the LDG network, and a reported body-wave magnitude $m_b$ from the ISC network.

In [ ]:
ldg_ml_isc_mb, query1_cat = cqt.get_agency_magnitude_pairs(db1, ("LDG", "ML"), ("ISC", "mb"), no_case=True)
cqt.plot_catalogue_map(map_config, query1_cat)

To visualise the comparisons we can either plot a conventional scatter plot or a density plot

In [ ]:
_ = cqt.plot_agency_magnitude_pair(ldg_ml_isc_mb)
_ = cqt.plot_agency_magnitude_density(ldg_ml_isc_mb, lognorm=False)

### Let's take a look at local magnitude scales

#### Comparing LDG and ROM

In [ ]:
# Query the catalogue
ldg_ml_rom_ml, query3_cat = cqt.get_agency_magnitude_pairs(db1, ("LDG", "ML"), ("ROM", "ML"), no_case=True)
# Plot the density of event pairs
_ = cqt.plot_agency_magnitude_density(ldg_ml_rom_ml)
# Map the pairs
cqt.plot_catalogue_map(map_config, query3_cat)

#### Comparing LDG and SED

In [ ]:
ldg_ml_zur_rmt, query4_cat = cqt.get_agency_magnitude_pairs(db1, ("LDG", "ML"), ("ZUR_RMT", "Mw"), no_case=True)
cqt.plot_agency_magnitude_pair(ldg_ml_zur_rmt)
cqt.plot_catalogue_map(map_config, query4_cat)

### Exploring the Relationships between Magnitude Scales

Large databases of earthquake can provide an enormous quantity of information to help understand how different recording agencies and different magnitude scales compare.

Ultimately, however, we are interested in deriving clear and applicable empirical models that can be used to convert between scales.

For this purpose, a set of regession tools are built in. As both magnitude scales are commonly reported with uncertainty, we make use of Python's tools for <b>Orthogonal Distance Regression</b>

Orthogonal Distance Regression is a more generalised form of orthogonal regression that searches for an optimal fit of a functional form (of any shape - not just linear!) taking into account error in both variables.

Several different functional forms are coded into the toolkit and can be called by a keyword (plus initial estimate of parameters)

* `polynomial`  - Nth order polynomial of the form: $y = c_0 + c_1 \cdot x + c_2 \cdot x^2 + \ldots c_n \cdot x^n$

* `2segmentM#.#` - 2-segment linear model with a cross-over at magnitude #.#, i.e. 

$y = c_0 + c_1 \cdot x$  (for $x < M_C$),    $y = c_2 + c_3 \cdot x$ for ($x \geq M_C$) 

* `exponential` - Exponential model of the form: $y = e^{\left( {c_0 + c_1 \cdot x} \right)} + c_2$

* `piecewise` - N-segment linear model with cross-over magnitudes determined in the fitting process

In [ ]:
regressor1 = cqt.CatalogueRegressor(ldg_ml_zur_rmt)
results = regressor1.run_regression("polynomial", [0., 1.0])
regressor1.results.pprint()
regressor1.plot_model(overlay=False)
print "Model standard deviation is %s" % str(regressor1.standard_deviation)

In [ ]:
regressor1 = cqt.CatalogueRegressor(ldg_ml_zur_rmt)
results = regressor1.run_regression("2segmentM4.3", [1.0, 1.0, 1.0])
regressor1.results.pprint()
regressor1.plot_model(overlay=False)
print regressor1.standard_deviation

In [ ]:
regressor1b = cqt.CatalogueRegressor(ldg_ml_zur_rmt)
results = regressor1b.run_regression("exponential", [1.0, 1.0, 1.0])
regressor1b.results.pprint()
regressor1b.plot_model(overlay=False)
print regressor1b.standard_deviation

In [ ]:
ldg_ml_csem_ml, query3_cat = cqt.get_agency_magnitude_pairs(db1, ("LDG", "ML"), ("CSEM", "ML"), no_case=True)
regressor2 = cqt.CatalogueRegressor(ldg_ml_csem_ml)
results = regressor2.run_regression("polynomial", [0.0, 1.0])
regressor2.results.pprint()
regressor2.plot_model_density(overlay=False, sample=0)
print regressor2.standard_deviation

In [ ]:
ldg_ml_str_ml, query4_cat = cqt.get_agency_magnitude_pairs(db1, ("LDG", "ML"), ("STR", "ML"), no_case=True)
regressor2 = cqt.CatalogueRegressor(ldg_ml_csem_ml)
results = regressor2.run_regression("polynomial", [0.0, 1.0])
regressor2.results.pprint()
regressor2.plot_model_density(overlay=False, sample=0)
print regressor2.standard_deviation

In [ ]:
str_ml_zur_mw, query4_cat = cqt.get_agency_magnitude_pairs(db1, ("STR", "ML"), ("ZUR_RMT", "Mw"), no_case=True)
regressor4 = cqt.CatalogueRegressor(str_ml_zur_mw)
results = regressor4.run_regression("polynomial", [0.0, 1.0])
regressor4.results.pprint()
regressor4.plot_model(overlay=False)
print regressor4.standard_deviation

In [ ]:
zur_mw_med_mw, query4_cat = cqt.get_agency_magnitude_pairs(db1, ("ZUR_RMT", "Mw"), ("MED_RCMT", "Mw"), no_case=True)
regressor4 = cqt.CatalogueRegressor(zur_mw_med_mw)
results = regressor4.run_regression("polynomial", [0.0, 1.0])
regressor4.results.pprint()
regressor4.plot_model(overlay=False)
print regressor4.standard_deviation

## Catalogue Homogenisation

The homogenisation process applies a set of conversion formulae to convert from each individual scale found in the database into a target magnitude scale defined by the user.

The user may wish to use the models from the regressions - or specify their own

The user must supply the hierarchy (in terms of the location and magnitude) by which certain conversion models may be applied. The tool will apply the conversion rule in order of preference.

The hierarchies may be changed for different time periods, different depths or different ``keys`` (the key in this case may correspond to a particular geographical region)



### Defining the Hierarchies

In the current example we build a catalogue by harmonising data from four agencies

<b>Agencies</b>:

For the period 1900 to 1990: LDG, STR, ISC, ROM, MDD, NEIC
For the period 1900 to 2014: STR, LDG, ROM, ISC, NEIC, MDD

<b>Magnitudes (harmonising to Moment Magnitude)</b>:

For the period 1900 to 1976: GCMT, HRVD, ZUR_RMT, MED_RCMT, STR (ML), LDG (ML), ISC (Ms), (ISC mb)

In [ ]:
from eqcat.isc_homogenisor import MagnitudeConversionRule, DynamicHomogenisor, HomogenisorPreprocessor

In [ ]:
origin_rules = [
    ("1900/01/01 - 1989/12/31", ["LDG", "STR", "ISC", "ROM", "MDD", "NEIC"]),
    ("1990/01/01 - 2014/12/31", ["STR", "LDG", "ROM", "ISC", "NEIC", "MDD"])
]

Each magnitude conversion rule is input as a Python function

In [ ]:
# Magnitude rules
def gcmt_hrvd(magnitude):
    return magnitude

def gcmt_hrvd_sigma(magnitude):
    return 0.0

def zur_med_rcmt(magnitude):
    return magnitude

def zur_med_rcmt_sigma(magnitude):
    """
    Adds uncertainty
    """
    return 0.1

def str_ml(magnitude):
    """
    From regression
    """
    return 0.320 + 0.858 * magnitude

def str_ml_sigma(magnitude):
    """
    From regression
    """
    return 0.21

def ldg_ml(magnitude):
    """
    Convert from LDG ML to Zurich CMT
    From regression 2 - segment
    """
    if magnitude < 4.3:
        return -0.397 + 0.995 * magnitude
    else:
        return 0.594 + 0.765 * magnitude

def ldg_ml_sigma(magnitude):
    """
    """
    if magnitude < 4.3:
        return 0.214
    else:
        return 0.284

def isc_ms(magnitude):
    """
    From Scordilis (2006)
    """
    if magnitude < 6.1:
        return 0.67 * magnitude + 2.07
    else:
        return 0.99 * magnitude + 0.08

def isc_ms_sigma(magnitude):
    """
    From Scordilis 2006
    """
    if magnitude < 6.1:
        return 0.17
    else:
        return 0.2

def isc_mb(magnitude):
    """
    From Scordilis (2006)
    """
    return 0.85 * magnitude + 1.03

def isc_mb_sigma(magnitude):
    """
    From Scordilis 2006
    """
    return 0.29    

To build the rules themselves, however, we use a class called ``MagnitudeConversionRule``, which stores information about the rule, i.e. to which agency it should be applied, to which magnitude scale, how to propagate uncertainty etc.

In [ ]:
magnitude_rule_set = [
    MagnitudeConversionRule("GCMT", "Mw", gcmt_hrvd, gcmt_hrvd_sigma),
    MagnitudeConversionRule("HRVD", "Mw", gcmt_hrvd, gcmt_hrvd_sigma),
    MagnitudeConversionRule("ZUR_RMT", "Mw", zur_med_rcmt, zur_med_rcmt_sigma),
    MagnitudeConversionRule("MED_RCMT", "Mw", zur_med_rcmt, zur_med_rcmt_sigma),
    MagnitudeConversionRule("STR", "ML", str_ml, str_ml_sigma),
    MagnitudeConversionRule("LDG", "ML", ldg_ml, ldg_ml_sigma),
    MagnitudeConversionRule("ISC", "Ms", isc_ms, isc_ms_sigma),
    MagnitudeConversionRule("ISC", "mb", isc_mb, isc_mb_sigma)
]

magnitude_rules = [
    ("1900/01/01 - 2015/12/31", magnitude_rule_set)
]

### Homogenise the Catalogue

As a pre-processing step we apply a tool that can search through the database and see which, if any, rule can be
applied to each earthquake

Those earthquakes for which no selection rule can be applied are neglected in the homogenisation process.


In this case we are expecting that the hierarchies changes with time - so we tell the preprocessing tool that the rules are time dependent.

In [ ]:
preprocessor = HomogenisorPreprocessor(rule_type="time")
preprocessed_catalogue = preprocessor.execute(catalogue, origin_rules, magnitude_rules)

##### Run the Homogenisation Tool ...

The logging can be useful to understand which rule (and which hierarchy) was adopted for each event. Most of the time it is not necessary, but if you find strange results in the harmonised catalogue then it is helpful to check.

In [ ]:
homogenisor = DynamicHomogenisor(preprocessed_catalogue, logging=True)

In [ ]:
output_catalogue = homogenisor.homogenise(magnitude_rules, origin_rules)

If we wish, we can export the log to a csv file

In [ ]:
homogenisor.dump_log("output_data/homogenisation1_logfile.csv")

### Finally ... we export the homogenised catalogue

In [ ]:
homogenisor.export_homogenised_to_csv("output_data/output_catalogue.csv")